## takes ~2 days (CPU only) to fine-tune mT5 with 7000 sanskrit sentences

In [1]:
!pip install -q transformers datasets sentencepiece
# sentencepiece installation will require restarting kernel after installation to take effect

In [2]:
import sentencepiece
print(sentencepiece.__version__)


0.2.0


In [3]:
from transformers import (
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset, Dataset
import numpy as np
import torch
import re
from transformers import AutoTokenizer

# Load mT5 Model and Tokenizer
# ===============================
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
MAX_LEN = 512

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
# ===============================
# Load and Clean Sanskrit Dataset
# ===============================
# Load and preprocess the dataset
dataset = load_dataset('oscar', 'unshuffled_deduplicated_hi', split='train[:1%]')


In [52]:

def clean_sanskrit_text(example):
    text = example["text"]
    
    # Remove zero-width characters and extra spaces
    text = re.sub(r'[\u200b-\u200d]', '', text)         # Remove zero-width characters
    text = re.sub(r'\s+', ' ', text)                    # Collapse multiple spaces/newlines
    text = text.strip()                                 # Trim leading/trailing whitespace
     # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove non-Sanskrit characters (retain Devanagari script)
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)
    
    return {"text": text}

dataset = dataset.map(clean_sanskrit_text)

def filter_by_word_count(example):
    word_count = len(example['text'].split())
    return 150 < word_count < 350

dataset = dataset.filter(filter_by_word_count)
print(dataset[0])

Map:   0%|          | 0/7594 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7594 [00:00<?, ? examples/s]

{'id': 26, 'text': 'देश के कम से कम राज्यों और दो केंद्र शासित प्रदेशों में हुए रेड अलर्टजी हां मौसम विभाग के मुताबिक सोमवार को आंधीतूफान भारी बारिश और ओलावृष्टि होने की हैं सम्भावना वही गृह मंत्रालय ने भी जानकारी देते हुए कहा हैं कि मौसम विभाग ने अलर्ट जारी किया हैं कि कश्मीर और हिमाचल प्रदेश के कुछ स्थानों पर सोमवार को आंधीतूफान और बारिश के साथ ओलावर्ष्टि हो सकती हैवहीं उत्तराखंड और पंजाब के कुछ स्थानों पर भारी बारिश और तेज हवाओं की संभावना हैं मौसम विभाग द्वारा जारी अलर्ट के बाद हरियाणा के सभी सरकारी और निजी स्कूलों को और मई को बंद रखने की घोषणा की गई हैं भारतीय मौसम विभाग के एक परामर्श का उल्लेख करते हुए गृह मंत्रालय के एक अधिकारी ने बताया कि असम मेघालय नगालैंड मणिपुर मिजोरम और त्रिपुरा के कुछ स्थानों पर भारी बारिश हो सकती है । वही अधिकारी ने यह भी बताया हैं कि जम्मूकश्मीर उत्तराखंड हिमाचल प्रदेश पंजाब हरियाणा चंडीगढ़ दिल्ली व एनसीआर और उत्तर प्रदेश के कुछ स्थानों पर भी बादल गरजने व बारिश होने और तेज हवाएं चलने की आशंका है इस रेड अलर्ट के जारी होने के बाद हम सभी की जिम्मेदारी बनती 

In [53]:
len(dataset)

5439

In [54]:
dataset[1]

{'id': 30,
 'text': 'सींग का नशे में चचेरे भाई बहन अंजलि ने मेरे दोस्त त्रिगुट और मीसेक्स सेक्स के साथ पार्टी के बाद बकवास किया सींग का नशे में चचेरे भाई बहन अंजलि ने मेरे दोस्त त्रिगुट और मीसेक्स सेक्स के साथ पार्टी के बाद बकवास किया भारतीय सेक्स सींग का नशे में चचेरे भाई बहन अंजलि ने मेरे दोस्त त्रिगुट और मीसेक्स सेक्स के साथ पार्टी के बाद बकवास किया सींग का नशे में चचेरे भाई बहन अंजलि ने मेरे दोस्त त्रिगुट और मीसेक्स सेक्स के साथ पार्टी के बाद बकवास किया की छवियां सींग का नशे में चचेरे भाई बहन अंजलि ने मेरे दोस्त त्रिगुट और मीसेक्स सेक्स के साथ पार्टी के बाद बकवास किया वीडियो विवरण सींग का नशे में चचेरे भाई बहन अंजलि ने मेरे दोस्त त्रिगुट और मीसेक्स सेक्स के साथ पार्टी के बाद बकवास किया आप अश्लील वीडियो देख रहे हैं सींग का नशे में चचेरे भाई बहन अंजलि ने मेरे दोस्त त्रिगुट और मीसेक्स सेक्स के साथ पार्टी के बाद बकवास किया। यह वीडियो सबसे बड़ी अश्लील वेबसाइट भारतीय सेक्स पर पोस्ट किया गया है। वीडियो की लंबाई के बारे में है। आप खोज बॉक्स का उपयोग करके अधिक गर्म अश्लील वीडियो खोज सकते है

In [55]:
dataset

Dataset({
    features: ['id', 'text'],
    num_rows: 5439
})

In [56]:
dataset[46]

{'id': 193,
 'text': 'तेहरान। दक्षिणपूर्वी ईरान में शुक्रवार को तीव्रता के भूकंप के कारण एक महिला की मौत हो गई और कई गांवों में कई घर क्षतिग्रस्त हो गए। ईरान के सरकारी मीडिया ने यह जानकारी दी है। रेड क्रिसेंट सोसायटी के प्रांतीय प्रमुख रसूल रश्की ने सरकारी संवाद समिति इरना को बताया कि भूकंप के बाद कई झटके महसूस किये गए और इसके कारण सदमे से एक वर्षीय महिला की मौत हो गई। रश्की ने बताया कि स्थानीय रिपोर्ट्स के अनुसार दो अन्य लोगों की मौत हुई है जिसकी तत्काल पुष्टि नहीं हो सकी है। सरकारी टेलीविजन ने आपदा प्रबंधन के प्रांतीय प्रमुख अब्देल रहमान शाहनवाजी के हवाले से बताया कि कई गांवों में कुछ घर क्षतिग्रस्त हुए हैं। अमेरिकी भूगर्भ सर्वेक्षण संस्थान ने बताया कि भूकंप की तीव्रता रिक्टर स्केल पर मापी गई और इसका केंद्र जेहादन शहर से किलो मीटर दक्षिण पश्चिम में जमीन से किलो मीटर की गहराई पर था। हर बार पार्टी में आप एक ही तरह का हेयर स्टाइल रखकर बोर हो गई हैं और अब तो आपके पति भी आपसे कहने लग पीएम मोदी ने वाराणसी में किया देश के पहले मल्टी मॉडल टर्मिनल का उद्घाटन अब गंगा के रास्ते चलेंगे जहाज बीजे

In [57]:
# 5. Tokenize the Dataset
# ===============================
max_input_length = MAX_LEN
max_target_length = MAX_LEN

def preprocess(example):
    input_text = example["text"]
    input_ids = tokenizer(
        input_text,
        padding="max_length",
        truncation=True,
        max_length=max_input_length
    )

    labels = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=max_target_length
    )

    input_ids["labels"] = labels["input_ids"]
    return input_ids

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])


Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

In [58]:
def preprocess_completion(examples):
    input_texts = ["complete: " + text[:145] for text in examples["text"]]
    target_texts = [ text[145:] for text in examples["text"]]  # dummy completion
    model_inputs = tokenizer(input_texts, padding="max_length", truncation=True, max_length=max_input_length,  return_attention_mask=True, 
                             add_special_tokens=True)
    
    #print(input_texts[0], " --- ", target_texts[0])
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_texts, padding="max_length", truncation=True, max_length=max_target_length,  return_attention_mask=True, 
                             add_special_tokens=False)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [59]:
# Shared training arguments
def get_training_args(output_dir):
    return Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=2,
        num_train_epochs=2,
        learning_rate=5e-4,
        logging_dir="./logs",
        logging_steps=5000,
        save_steps=5000,
        save_total_limit=1,
        predict_with_generate=True,
        fp16=False
    )

# Trainer setup
def train_model(preprocess_fn, output_dir, remove_cols):
    tokenized_dataset = dataset.map(preprocess_fn, batched=True, remove_columns=remove_cols)
    training_args = get_training_args(output_dir)
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
    )
    trainer.train()

# Task-specific training functions
def train_completion():
    print("Training for text completion...")
    train_model(preprocess_completion, "./mt5-sanskrit-completion", remove_cols=["text"])

# Evaluation functions
def evaluate(task, test_input):
    input_ids = tokenizer(test_input, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids, max_length=512)
    print(f"\n{task} Result:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


In [60]:
train_completion()

Training for text completion...


Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

/var/tmp/ipykernel_6740/255034336.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss


In [61]:
'''
from transformers import (
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset, Dataset
import numpy as np
import torch
import re
from transformers import AutoTokenizer

checkpoint_dir = "mt5-sanskrit-completion/checkpoint-5000/"  # your saved checkpoint directory

tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
model = MT5ForConditionalGeneration.from_pretrained(checkpoint_dir)
'''

'\nfrom transformers import (\n    MT5ForConditionalGeneration,\n    MT5Tokenizer,\n    Seq2SeqTrainer,\n    Seq2SeqTrainingArguments,\n    DataCollatorForSeq2Seq\n)\nfrom datasets import load_dataset, Dataset\nimport numpy as np\nimport torch\nimport re\nfrom transformers import AutoTokenizer\n\ncheckpoint_dir = "mt5-sanskrit-completion/checkpoint-5000/"  # your saved checkpoint directory\n\ntokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)\nmodel = MT5ForConditionalGeneration.from_pretrained(checkpoint_dir)\n'

In [ ]:
evaluate("Text Completion", "आज सुबह जब मैं पार्क गया, तो मैंने देखा कि")


Text Completion Result:
 ा यहाँ ही हमने देखा कि मैं उसकी बात और बात थी कि मैं उसकी बात और बात थी कि मैंने उसकी बात और बात ही थी कि मैंने उसकी बात और उसकी बात और उसकी बात थी कि मैं उसकी बात कर ली है कि मैंने उसकी बात और उसकी बात और बात ही जिसकी बात हमेशा से ही हमने हमने कहा कि मैंने उसकी बात और बातचीत में हमारे बारे में बात करते हुए बताया कि मैंने उसकी बात बात करते हुए उसने बताया कि मैं उसकी बात सुनते हुए हमने बताया कि मैंने हमारे बारे में बताया कि मैंने उसकी बात करते हुए मैं उसकी बात और उसकी बात और उसकी बात करते हुए हमने उसकी बात सुनकर हमने बताया कि मैंने उसकी बात की बात करते थे कि मैंने उसकी बात नही कर पाई और मैं उसकी बात बात और हमेशा हमने बात करते हुए मैं यही बात करते हुए मैंने उसकी बात करते हुए जिसमें मैं उसकी बात और उसकी बात सुनते हुए उसकी बात ही यह बात करते हुए मैं उसकी बात करतें थीं मैंने उसकी बात और मैंने ही बात करते हुए उसकी बात और बात करते हुए उसकी बात और उसकी बात और वो बात करते हुए नजर आते हुए और मैंने बात करते हुए हमने यह बात करते हुए हम बात करते हुए कहा कि मैंने यह बात हमा

# but fine-tuning for 1 day (and 5000 sentences) insufficient to complete sentences for unseen data

In [ ]:
evaluate("Text Completion", "वह बच्चा स्कूल से लौटते समय")


Text Completion Result:
 े ही स्कूल से लौटते हुए लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते समय स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते समय स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल के लौटते समय स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल के लौटते हुए स्कूल से लौटने लग रहे स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटने के बाद स्कूल से लौटते हुए स्कूल से लौटते हुए स्कूल से लौटते समय स्कूल से लौटते हु


In [ ]:
evaluate("Text Completion", "बारिश इतनी तेज हो रही थी कि लोग")


Text Completion Result:
 कर रही थी। जबकि लोगों ने यहां से ही ही बात कर लिया है। लोगों ने यहां तक ही यह बात करने की बात थी। जबकि लोगों ने यह बात करते हुए उसने यह जान किया था। जिससे लोगों ने उसकी बातें तो यह भी कहा कि जब लोगों ने यह बताया कि हमने तक लोगों ने यह बताया कि वह अपने पास ही ही बात कर लिया था। जबकि लोगों ने यह जानना चाहते थे कि वह उसकी वजह से ही यह जानकारी दी। यह जानकारी यह है कि वह हमारे पास ही यह जानकारी दी जिससे हमारे पास थोड़ी तेज हो गई है। यहां ही लोगों ने उसने बात करते हुए कहा कि हमारे पास ही उसने यह जानकारी दी। जिससे वो यह जानकारी दी थी कि वह लोगों को यह जानना चाहते हैं कि वह लोगों को यह बात करते हुए यह जानकारी दी थी कि वह यहां से बात कराया और वह नजर आए। उसने बात करते हुए कहा कि वो उसने ही यह जानकारी दी। जिससे वह उसकी बात करते हुए यह जानकारी दी थी कि वह यहां ही ही उसने यह जानकारी दी थी कि लोगों को यह जानकारी दी। जिसकी वजह से यह बात ही बात करते हुए उसने बातचीत करते हुए यह बात कर रही थी कि यह जानकारी दी थी कि वो जिसमें उसने हमारे पास ही यह जानकारी दी है कि वह बात करते हुए

In [ ]:
evaluate("Text Completion", "बचपन की सबसे प्यारी याद मेरे लिए")


Text Completion Result:
 ा रहा और अब तक ही तुम ही याद ही तुम ही याद ही वो याद ही ही तुम्हारी याद मेरे लिए याद ही हम आपके लिए हम आपको ही बताए ही वो हम आपको याद ही हम आपको याद ही हमेशा याद ही हम आपके लिए हमने हम आपको ही प्यारी याद मेरे लिए हमारे लिए हम आपको ही प्यारी ही याद रखते हैं हम आपको अपनी याद ही याद ही याद ही हम आपको याद ही हम आपको ही याद ही तुम्हारी याद रखने की सबसे प्यारी याद ही याद ही तुम्हारी याद मेरी याद ही हम आपके लिए याद ही हम आपके लिए नई बात है हम आपके लिए हम आपको याद रखता है हम आपको ही हम आपको याद किया हम आपके लिए याद ही याद रखें हम आपको ही याद रखें हम आपको हमेशा ही प्यारी याद हम आपके लिए ही याद रखें हम आपके लिए प्यारी याद रखें हम आपको ही याद रखें हम आपके लिए हम आपके लिए हम सबसे पसंद जायेगा हम आपको याद रखें हम आपको हमारे लिए हम आपको यह बात हम आपको याद रखें हम आपको पसंद है हम आपके लिए हम आपको ही प्यारी याद हम ही हम आपके लिए हम आपको याद रखता है हम आपको याद ही हम आपके लिए यही हमेशा ही याद हम आपके लिए हम आपके लिए हमेशा ही प्यारी याद हम आपको याद रखें हम आपके पास ही प्यारी न ह

In [ ]:
evaluate("Text Completion", "वनवास के समय श्रीराम ने माता सीता और लक्ष्मण के साथ मिलकर")


Text Completion Result:
 चिपक रही थी और उसने उस और उसने साथ ही ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ दिया। श्रीराम ने माता और साथ साथ साथ ही साथ ही साथ साथ ही साथ ही साथ ही साथ ही श्रीराम ने माता और साथ ही ही साथ साथ साथ ही साथ ही साथ ही साथ ही साथ ही साथ साथ ही साथ ही साथ ही साथ साथ ही ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ साथ ही साथ ही साथ साथ ही साथ साथ ही साथ ही ही साथ ही साथ ही साथ ही साथ साथ ही साथ ही साथ ही साथ ही साथ ही साथ साथ ही साथ ही साथ ही साथ साथ साथ ही साथ ही साथ ही श्रीराम ने साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही ही साथ ही साथ ही साथ ही साथ साथ ही साथ साथ ही साथ ही ही साथ साथ ही साथ ही साथ ही साथ ही ही साथ ही साथ ही साथ ही साथ ही ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही ही साथ ही साथ ही साथ साथ ही साथ ही ही साथ साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ ही साथ 


In [ ]:
!rm -r logs 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
